In [30]:
import pandas as pd
import numpy as np
from flask import Flask, render_template, jsonify

In [3]:
circuits_df = pd.read_csv("datasets/circuits.csv")
circuits_df

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,\N,http://en.wikipedia.org/wiki/Losail_Internatio...


In [9]:
constructor_results_df = pd.read_csv('datasets/constructor_results.csv')
constructor_results_df

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N
...,...,...,...,...,...
12285,16785,1110,214,10.0,\N
12286,16786,1110,213,1.0,\N
12287,16787,1110,51,0.0,\N
12288,16788,1110,3,0.0,\N


In [11]:
constructor_standings_df = pd.read_csv('datasets/constructor_standings.csv')
constructor_standings_df

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0
...,...,...,...,...,...,...,...
13046,28568,1110,214,57.0,6,6,0
13047,28569,1110,3,11.0,7,7,0
13048,28570,1110,213,3.0,10,10,0
13049,28571,1110,210,11.0,8,8,0


In [14]:
constructors_df = pd.read_csv('datasets/constructors.csv')
constructors_df

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri


In [41]:
results = pd.read_csv('datasets/results.csv')
races = pd.read_csv('datasets/races.csv')
drivers = pd.read_csv('datasets/drivers.csv')
constructors = pd.read_csv('datasets/constructors.csv')

# Merge datasets to get required data
# data_merged = results.merge(races, on='raceId').merge(drivers, on='driverId').merge(constructors, on='constructorId', )
data_merged = results.merge(races[['raceId', 'year']], on='raceId', suffixes=('', '_race'))
data_merged = data_merged.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', suffixes=('', '_driver'))
data_merged = data_merged.merge(constructors[['constructorId', 'name']], on='constructorId', suffixes=('', '_constructor'))

# Filter data for years 2010 onwards
data_merged = data_merged[data_merged['year'] >= 2010]

# Aggregate points by driver across all years and select top 10 drivers
top_drivers = data_merged.groupby(['driverId', 'forename', 'surname'])['points'].sum().nlargest(10).reset_index()['driverId']

# Filter the merged data to include only top 10 drivers
data_merged = data_merged[data_merged['driverId'].isin(top_drivers)]

In [42]:
data_merged.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'forename', 'surname', 'name'],
      dtype='object')

In [43]:
data_merged

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,forename,surname,name
52,20325,337,1,1,2,4,3,3,3,15.0,...,5983578,42,4,1:59.560,189.665,1,2010,Lewis,Hamilton,McLaren
53,20352,338,1,1,2,11,6,6,6,8.0,...,5646429,47,3,1:28.506,215.700,1,2010,Lewis,Hamilton,McLaren
54,20376,339,1,1,2,20,6,6,6,8.0,...,5651883,32,4,1:37.745,204.151,1,2010,Lewis,Hamilton,McLaren
55,20396,340,1,1,2,6,2,2,2,18.0,...,6403693,13,1,1:42.061,192.273,1,2010,Lewis,Hamilton,McLaren
56,20432,341,1,1,2,3,14,14,14,0.0,...,\N,59,1,1:24.357,198.655,29,2010,Lewis,Hamilton,McLaren
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25816,24923,1045,815,211,11,5,18,18,18,0.0,...,\N,40,9,1:33.629,208.089,5,2020,Sergio,Pérez,Racing Point
25817,24926,1046,815,211,11,5,1,1,1,25.0,...,5475114,69,3,0:56.789,224.599,1,2020,Sergio,Pérez,Racing Point
25818,24965,1047,815,211,11,19,\N,R,20,0.0,...,\N,6,19,1:43.263,193.625,7,2020,Sergio,Pérez,Racing Point
25856,26058,1109,817,213,3,13,13,13,13,0.0,...,\N,67,17,1:23.934,187.904,11,2023,Daniel,Ricciardo,AlphaTauri


In [44]:
data_sankey = data_merged.groupby(['forename', 'surname', 'name', 'year'])['points'].sum().reset_index()
data_sankey['source'] = data_sankey['forename'] + ' ' + data_sankey['surname']
data_sankey['target'] = data_sankey['name']
data_sankey['value'] = data_sankey['points']
nodes = list(set(data_sankey['source'].tolist() + data_sankey['target'].tolist()))
links = [
    {"source": nodes.index(row['source']), "target": nodes.index(row['target']), "value": row['value'], "year": row['year']}
    for index, row in data_sankey.iterrows()
]

In [49]:
data_sankey[data_sankey['surname']=='Vettel']

,forename,surname,name,year,points,source,target,value
72,Sebastian,Vettel,Aston Martin,2021,43.0,Sebastian Vettel,Aston Martin,43.0
73,Sebastian,Vettel,Aston Martin,2022,37.0,Sebastian Vettel,Aston Martin,37.0
74,Sebastian,Vettel,Ferrari,2015,278.0,Sebastian Vettel,Ferrari,278.0
75,Sebastian,Vettel,Ferrari,2016,212.0,Sebastian Vettel,Ferrari,212.0
76,Sebastian,Vettel,Ferrari,2017,317.0,Sebastian Vettel,Ferrari,317.0
77,Sebastian,Vettel,Ferrari,2018,320.0,Sebastian Vettel,Ferrari,320.0
78,Sebastian,Vettel,Ferrari,2019,240.0,Sebastian Vettel,Ferrari,240.0
79,Sebastian,Vettel,Ferrari,2020,33.0,Sebastian Vettel,Ferrari,33.0
80,Sebastian,Vettel,Red Bull,2010,256.0,Sebastian Vettel,Red Bull,256.0
81,Sebastian,Vettel,Red Bull,2011,392.0,Sebastian Vettel,Red Bull,392.0
